In [7]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tempfile
import pymupdf
import re
from dotenv import load_dotenv
import google.generativeai as genai
import time
import ast

In [8]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

In [ ]:
filtrados_true = pd.read_excel('../docs/respostas_danoambiental_50.xlsx')
filtrados_true.head(10)

,Unnamed: 0,processo,processo_ambiental,justificativa,processoAnexoID,link de referencia
0,0000010-75.2024.8.02.0356,0000010-75.2024.8.02.0356,False,O texto descreve uma ação judicial por danos m...,1676545211,http://jud-anexos.digesto.com.br/c1541200bd444...
1,0000135-52.2021.8.16.0067,0000135-52.2021.8.16.0067,True,Trata-se de Ação Civil Pública ajuizada pelo M...,2908924070,http://jud-anexos.digesto.com.br/a5625273387d0...
2,0000156-20.2023.8.02.0076,0000156-20.2023.8.02.0076,False,O texto trata de uma ação de indenização por d...,1419499656,http://jud-anexos.digesto.com.br/25afbf7fded56...
3,0000272-92.1992.8.26.0157,0000272-92.1992.8.26.0157,True,Ação Civil Pública - Meio Ambiente,657568057,http://jud-anexos.digesto.com.br/4f696a2b03b85...
4,0000428-83.2019.8.16.0134,0000428-83.2019.8.16.0134,True,Assunto Principal: Dano Ambiental,1088560478,http://jud-anexos.digesto.com.br/05d31619ede92...
5,0000527-75.2011.8.02.0020,0000527-75.2011.8.02.0020,True,"Na petição inicial, dentre seus pedidos, os au...",1186043580,http://jud-anexos.digesto.com.br/7487b4175300b...
6,0000627-51.2015.8.16.0068,0000627-51.2015.8.16.0068,False,Assunto Principal: Indenização por Dano Moral,1397328389,http://jud-anexos.digesto.com.br/6b0828e58be5a...
7,0000640-15.2017.8.11.0102,0000640-15.2017.8.11.0102,True,O Ministério Público do Estado de Mato Grosso ...,1299854564,http://jud-anexos.digesto.com.br/40b55644f0bfc...
8,0001293-11.2021.8.16.0046,0001293-11.2021.8.16.0046,True,Trata-se de “ação civil pública por dano ambie...,1462784031,http://jud-anexos.digesto.com.br/a7e7d6002b57d...
9,0001700-76.2018.8.17.3220,0001700-76.2018.8.17.3220,False,O texto trata de ação declaratória de inexistê...,1484217656,http://jud-anexos.digesto.com.br/6a3d66555a358...


In [ ]:
filtrados_com_num = pd.read_excel("../docs/processos_filtrados.xlsx")
filtrados_com_num.head(10)

,Unnamed: 0,CNJ,processoID,processoAnexoID,Download copia,Download original,Tipo de anexo,Publicado em,Obtido em,Tribunal,Titulo,num_processo
0,0,0800895-53.2022.8.10.0049,652049379,2643544408,http://jud-anexos.digesto.com.br/9d04a7ce4bf3c...,https://pje.tjma.jus.br/pje/Painel/painel_usua...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL PDF 636 64 KB,0800895-53.2022.8.10.0049
1,1,0800895-53.2022.8.10.0049,652049379,2643544419,http://jud-anexos.digesto.com.br/e512fdd683d21...,https://pje.tjma.jus.br/pje/Painel/painel_usua...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL,0800895-53.2022.8.10.0049
2,2,0800895-53.2022.8.10.0049,652049379,2643544426,http://jud-anexos.digesto.com.br/80b47c8d75bae...,https://pje.tjma.jus.br/pje/Processo/ConsultaP...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL,0800895-53.2022.8.10.0049
3,3,5003677-41.2019.8.13.0521,612465137,590939495,http://jud-anexos.digesto.com.br/588fa00346ccf...,https://pje.tjmg.jus.br/pje/Painel/painel_usua...,LAUDO,2020-09-06,2024-06-28T13:05:15.036135,TJMG,LAUDO PERICIAL - LAUDO PERICIAL PDF 1 437 88 KB,5003677-41.2019.8.13.0521
4,4,5003677-41.2019.8.13.0521,612465137,590939496,http://jud-anexos.digesto.com.br/f262789ad3d60...,https://pje.tjmg.jus.br/pje/Painel/painel_usua...,LAUDO,2020-09-06,2024-06-28T13:05:15.036135,TJMG,LAUDO PERICIAL - LAUDO PERICIAL,5003677-41.2019.8.13.0521
5,5,5003677-41.2019.8.13.0521,612465137,590939497,http://jud-anexos.digesto.com.br/51644135dd7d3...,https://pje.tjmg.jus.br/pje/Processo/ConsultaP...,LAUDO,2020-09-06,2024-06-28T13:05:15.036135,TJMG,LAUDO PERICIAL - LAUDO PERICIAL,5003677-41.2019.8.13.0521
6,6,0000527-75.2011.8.02.0020,582564055,1186042345,http://jud-anexos.digesto.com.br/b9775f943d51d...,https://www2.tjal.jus.br/pastadigital/getPDF.d...,LAUDO,2016-08-17,2024-09-12T04:13:58.579221,TJAL,LAUDO DE AVALIACAO - PAGINA 99,0000527-75.2011.8.02.0020
7,7,0011297-52.2011.8.13.0140,690025576,2589045924,http://jud-anexos.digesto.com.br/6a7cee8b71f2a...,https://pje.tjmg.jus.br/pje/Processo/ConsultaP...,LAUDO,2023-08-09,2025-02-21T07:26:39.772134,TJMG,LAUDO PERICIAL - PARTE 01 - LAUDO PERICIAL CAR...,0011297-52.2011.8.13.0140
8,8,0011297-52.2011.8.13.0140,690025576,2589045929,http://jud-anexos.digesto.com.br/0584da20569c9...,https://pje.tjmg.jus.br/pje/Processo/ConsultaP...,LAUDO,2023-08-09,2025-02-21T07:26:39.772134,TJMG,LAUDO DE PERICIA - PARTE 02 - LAUDO OFICIAL CA...,0011297-52.2011.8.13.0140
9,9,0011297-52.2011.8.13.0140,690025576,2589045772,http://jud-anexos.digesto.com.br/9d774ad2a90a2...,https://pje.tjmg.jus.br/pje/Processo/ConsultaP...,LAUDO,2024-01-25,2025-02-21T07:26:39.772134,TJMG,LAUDO - LAUDO COMPLEMENTAR CARMENSE COMERCIAL ...,0011297-52.2011.8.13.0140


In [11]:
processos_unicos = list(filtrados_com_num['num_processo'].unique())
display(filtrados_com_num.loc[filtrados_com_num['num_processo']=='Não encontrado'])
processos_unicos.remove('Não encontrado')

,Unnamed: 0,CNJ,processoID,processoAnexoID,Download copia,Download original,Tipo de anexo,Publicado em,Obtido em,Tribunal,Titulo,num_processo
51,51,5580808-19.2022.8.09.0011,503987107,972210470,http://jud-anexos.digesto.com.br/313133a43d0b9...,https://projudi.tjgo.jus.br/BuscaProcesso?Pagi...,DECISAO,2023-01-18,2024-08-23T05:34:28.806878,TJGO,DECISAO DE PEDIDO DE URGENCIA,Não encontrado
55,55,5061164-56.2024.8.09.0051,570015135,2564299827,http://jud-anexos.digesto.com.br/2a8bb47194987...,https://projudi-2024-prd.s3.tjgo.jus.br/202412...,DECISAO,2024-12-04,2025-02-19T07:58:29.294476,TJGO,DECISAO,Não encontrado
102,102,0000257-34.2024.8.05.0110,655333385,1942855138,http://jud-anexos.digesto.com.br/30f8d4e93b94a...,https://projudi.tjba.jus.br/projudi/listagens/...,DOCUMENTO,2024-02-26,2024-11-30T22:32:58.025278,TJBA,DOCUMENTO,Não encontrado


In [12]:
testes = filtrados_true[1:100]

In [13]:
processo_anexo_id_lista = testes['processoAnexoID'].tolist()

In [14]:
processo_anexo_id_lista

[2908924070,
 1419499656,
 657568057,
 1088560478,
 1186043580,
 1397328389,
 1299854564,
 1462784031,
 1484217656,
 2193427480,
 2924793560,
 374401292,
 2002955426,
 2583016248,
 2447744989,
 2890794934,
 2374549812,
 2451590480,
 2090274323,
 2229235049,
 2332883980,
 2887865420,
 2582226524,
 1249838163,
 1987957126,
 2431711293,
 1124178884,
 2722162734,
 1053877654,
 1148584788,
 2124556719,
 2589045540,
 2762584664,
 1182706696,
 2925804315,
 2924963112,
 2635090005,
 2718369649,
 2935865691,
 2075508189,
 1494896928,
 2015181677,
 2014041729,
 812664082,
 1923392718,
 2271997843,
 2685702174,
 2762585026,
 2447098711]

In [ ]:
def analise(texto_extraido,respostas,numero_processo,processoAnexoID,link):

    prompt = f"""
    Tarefa: Você receberá um texto bruto que descreve um processo judicial relacionado a algum dano ambiental. 
    Seu objetivo é analisar esse texto e extrair 13 informações específicas, conforme as instruções abaixo.
    Definição útil: Dano ambiental é qualquer prejuízo causado ao meio ambiente (água, solo, ar, fauna, flora, patrimônio paisagístico etc.) por ação ou omissão de um terceiro, sendo gerador de obrigação de reparação, conforme os artigos 186 e 927 do Código Civil.
    Retorno esperado (formato da resposta):
    A resposta deve ser apenas uma lista com 13 elementos, nesta ordem específica:

    [0] Georreferência do local afetado, no formato: XX°xx’xx.xx” S e XX°xx’xx.xx” O
     Se não houver, retorne: "Não encontrado georreferencia"

    [1] Unidade Federativa (UF) completa (ex: "Maranhão", "São Paulo" etc.)
     Se não houver, retorne: "Não encontrado UF"

    [2] Município ou cidade do local afetado.
     Se não houver, retorne: "Não encontrado cidade"

    [3] Nome do responsável pelo dano ambiental (empresa ou pessoa física).
     Se não houver, retorne: "Não encontrado responsavel"

    [4] Categoria do responsável: "Pessoa Física" ou "Pessoa Jurídica"
     Se não houver, retorne: "Não encontrado pessoa"

    [5] Data do impacto ambiental no formato: DD/MM/AA
     Se não houver, retorne: "Não encontrado data"

    [6] Extensão da área afetada (ex: "120 hectares", "15.000 m²", etc.)
     Se não houver, retorne: "Não encontrado área"

    [7] Unidade de medida da área (ex: "hectares", "m²", etc.)
     Se não houver, retorne: "Não encontrado medida"

    [8] Metodologia de punição aplicada (ex: auto de infração, processo administrativo, etc.)
     Se não houver, retorne: "Não encontrado método"

    [9] Compensação não monetária atribuída  (ou seja, alguma ação de reparação ambiental, como reflorestamento, recuperação de áreas degradadas, etc.)
     Se não houver, retorne: "Não encontrada compensação"

    [10] Categoria da compensação: "Multas Administrativas", "Compensações Financeiras", "Obrigações de Fazer (com custo)", " Custas Judiciais e Acordos" ou "Valoração Econômica".
     Se não houver, retorne: "Não encontrada categoria"

    [11] Tipo de Valor da compensação de acordo com a categoria da compensação: para multas administrativas existe 'Multa simples', 'Multa diária',entre outras. para compensações financeiras existe 'Indenização por dano material', 'Indenização por dano moral coletivo','Compensação ambiental (Lei do SNUC)' entre outros. Para obrigações de fazer (com custo) existe por exemplo 'Reflorestamento ou recomposição vegetal', 'Descontaminação de solo e água', entre outros. Para C'ustas Judiciais e Acordos'	existe 'Custas processuais e honorários' por exemplo. Para 'Valoração Econômica' existe	'Valor de mercado dos bens naturais afetados' por exemplo.
     Se não houver, retorne: "Não encontrado tipo valor"

    [12] Forma de Cálculo ou valor (por exemplo: R$ 5.000, R$ 500 a R$ 10 mil/dia., reflorestamento a R$ 30 mil/hectare, R$ 3 milhões para limpar lençol freático, 10% a 20% do valor da causa)

    IMPORTANTE: Não retorne o texto original, Não inclua explicações ou formatações extras, Apenas a lista com os 13 elementos na ordem acima, sem repetições. Não devolva o índice da lista, apenas os valores. Não inclua aspas ou colchetes na resposta. Se não encontrar algum elemento, retorne "Não encontrado" na posição correspondente da lista, apenas o valor da resposta.
    A resposta deve ser uma lista com EXATAMENTE 13 elementos (de índice 0 a 12), na ordem definida acima. Não retorne elementos extras. Não adicione vírgulas no final da lista. Não inclua o texto original, nem explicações, apenas a lista limpa.
    
    Texto bruto: {texto_extraido}"""
    generation_config_low_temp = genai.GenerationConfig(
        max_output_tokens=500,
        temperature=0.0,
)
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    resposta = model.generate_content(prompt, generation_config=generation_config_low_temp)
    time.sleep(3)
    lista = (resposta.text.split("\n"))
    lista.insert(0,numero_processo)
    lista = [x for x in lista if x != '']
    lista.append(processoAnexoID)
    lista.append(link)
    
    respostas[numero_processo] = lista
    return respostas
    

In [ ]:
respostas = {}
amostra_danos_ambientais = filtrados_true.loc[filtrados_true['processo_ambiental'] != False]

                   Unnamed: 0                   processo  processo_ambiental  \
1   0000135-52.2021.8.16.0067  0000135-52.2021.8.16.0067                True   
3   0000272-92.1992.8.26.0157  0000272-92.1992.8.26.0157                True   
4   0000428-83.2019.8.16.0134  0000428-83.2019.8.16.0134                True   
5   0000527-75.2011.8.02.0020  0000527-75.2011.8.02.0020                True   
7   0000640-15.2017.8.11.0102  0000640-15.2017.8.11.0102                True   
8   0001293-11.2021.8.16.0046  0001293-11.2021.8.16.0046                True   
11  0002010-52.2014.8.16.0148  0002010-52.2014.8.16.0148                True   
12  0002156-82.2011.8.16.0121  0002156-82.2011.8.16.0121                True   
14  0002171-93.2018.8.08.0035  0002171-93.2018.8.08.0035                True   
16  0002250-66.2015.8.16.0193  0002250-66.2015.8.16.0193                True   
17  0002587-84.2020.8.12.0001  0002587-84.2020.8.12.0001                True   
18  0002728-36.2017.8.14.0104  0002728-3

In [ ]:
for processo in amostra_danos_ambientais['processo']: 
    linhas_correspondentes = filtrados_com_num.loc[filtrados_com_num['num_processo'] == processo]
    texto = ''
   
    for index, row in linhas_correspondentes.iterrows():
        processoAnexoID = row['processoAnexoID']
        link = row['Download copia']
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        
        if 'pdf' in content_type:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
        
        elif 'html' in content_type:
            soup = BeautifulSoup(response.content, 'html.parser')
            texto += soup.get_text()

        respostas = analise(texto,respostas,processo,processoAnexoID,link)

Unnamed: 0                                                           6
CNJ                                          0000527-75.2011.8.02.0020
processoID                                                   582564055
processoAnexoID                                             1186042345
Download copia       http://jud-anexos.digesto.com.br/b9775f943d51d...
Download original    https://www2.tjal.jus.br/pastadigital/getPDF.d...
Tipo de anexo                                                    LAUDO
Publicado em                                                2016-08-17
Obtido em                                   2024-09-12T04:13:58.579221
Tribunal                                                          TJAL
Titulo                                  LAUDO DE AVALIACAO - PAGINA 99
num_processo                                 0000527-75.2011.8.02.0020
Name: 6, dtype: object
Unnamed: 0                                                           7
CNJ                                          0011297-5

KeyboardInterrupt: 

In [ ]:
respostas_teste1 = pd.DataFrame.from_dict(respostas, orient='index')

respostas_teste1.columns = [
    "processo",
    "georreferência",  
    "UF",
    "municipio",
    "responsavel",
    "tipo_responsavel",
    "data",
    "area",
    "unidade",
    "metodologia_punição",
    'compensacao_nao_monetaria',
    "categoria_compensacao",
    "tipo_valor",
    "calculo",
    "id_documento",
    "url_documento"
]

respostas_teste1

,processo,georreferência,UF,municipio,responsavel,tipo_responsavel,data,area,unidade,metodologia_punição,compensacao_nao_monetaria,categoria_compensacao,tipo_valor,calculo,id_documento,url_documento
0000527-75.2011.8.02.0020,0000527-75.2011.8.02.0020,Não encontrado georreferencia,Não encontrado UF,Não encontrado cidade,Não encontrado responsavel,Não encontrado pessoa,Não encontrado data,Não encontrado área,Não encontrado medida,Não encontrado método,Não encontrada compensação,Não encontrada categoria,Não encontrado tipo valor,Não encontrado,1186042345,http://jud-anexos.digesto.com.br/b9775f943d51d...
0011297-52.2011.8.13.0140,0011297-52.2011.8.13.0140,Não encontrado georreferencia,Minas Gerais,Carmo da Mata,Carmense Comercial Ltda,Pessoa Jurídica,04/05/2023,Não encontrado área,Não encontrado medida,Processo administrativo,Descomissionamento completo do parque industri...,Compensações Financeiras,Indenização por dano material,"R$ 2.447.933,38",2589045719,http://jud-anexos.digesto.com.br/387f948353a42...


In [ ]:
filtrados_true2 = pd.read_excel('../docs/respostas_danoambiental_51_100.xlsx')
filtrados_true2 = filtrados_true2.loc[filtrados_true2['processo_ambiental'] != False]

for processo in filtrados_true2['processo']: 
    linhas_correspondentes = filtrados_com_num.loc[filtrados_com_num['num_processo'] == processo]
    texto = ''
   
    for index, row in linhas_correspondentes.iterrows():
        processoAnexoID = row['processoAnexoID']
        link = row['Download copia']
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        
        if 'pdf' in content_type:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
        
        elif 'html' in content_type:
            soup = BeautifulSoup(response.content, 'html.parser')
            texto += soup.get_text()

        respostas = analise(texto,respostas,processo,processoAnexoID,link)

respostas_teste2 = pd.DataFrame.from_dict(respostas, orient='index')

respostas_teste2.columns = [
    "processo",
    "georreferência",  
    "UF",
    "municipio",
    "responsavel",
    "tipo_responsavel",
    "data",
    "area",
    "unidade",
    "metodologia_punição",
    'compensacao_nao_monetaria',
    "categoria_compensacao",
    "tipo_valor",
    "calculo",
    "id_documento",
    "url_documento"
]

In [41]:
respostas_teste2

,processo,georreferência,UF,municipio,responsavel,tipo_responsavel,data,area,unidade,metodologia_punição,compensacao_nao_monetaria,categoria_compensacao,tipo_valor,calculo,id_documento,url_documento
0000527-75.2011.8.02.0020,0000527-75.2011.8.02.0020,Não encontrado georreferencia,Não encontrado UF,Não encontrado cidade,Não encontrado responsavel,Não encontrado pessoa,Não encontrado data,Não encontrado área,Não encontrado medida,Não encontrado método,Não encontrada compensação,Não encontrada categoria,Não encontrado tipo valor,Não encontrado,1186042345,http://jud-anexos.digesto.com.br/b9775f943d51d...
0011297-52.2011.8.13.0140,0011297-52.2011.8.13.0140,Não encontrado georreferencia,Minas Gerais,Carmo da Mata,Carmense Comercial Ltda,Pessoa Jurídica,04/05/2023,Não encontrado área,Não encontrado medida,Processo Judicial Eletrônico,Descomissionamento completo do parque industri...,Obrigações de Fazer (com custo),"Reflorestamento ou recomposição vegetal, Desco...","R$ 3.807.896,40",2589045924,http://jud-anexos.digesto.com.br/6a7cee8b71f2a...
0600357-89.2021.8.04.5600,0600357-89.2021.8.04.5600,Não encontrado georreferencia,Amazonas,Manicoré,Destaque Indústria e Comércio de Madeiras Ltda,Pessoa Jurídica,30/04/2016,Não encontrado área,metros cúbicos,Processo administrativo,Indenização por danos materiais e dano moral c...,Compensações Financeiras,Indenização por dano material e Indenização po...,A ser quantificada em sede de cumprimento de s...,2344968075,http://jud-anexos.digesto.com.br/5ab34519c8f96...
0600384-72.2021.8.04.5600,0600384-72.2021.8.04.5600,07°43’11” S e 61°28’11” W,Amazonas,Manicoré,Carlos Alexandre Rossi,Pessoa Física,16/06/2016,"121,15",hectares,Auto de infração,Isolamento da área desmatada,Obrigações de Fazer (com custo),Isolamento da área,A ser quantificada em sede de cumprimento de s...,2294638453,http://jud-anexos.digesto.com.br/7abb2bb3a616f...
0600592-22.2022.8.04.5600,0600592-22.2022.8.04.5600,Não encontrado georreferencia,Amazonas,Manicoré,C R MADEIREIRA AMAZONAS LTDA-EPP,Pessoa Jurídica,07/11/2018,Não encontrado área,Não encontrado medida,Processo administrativo,Abstenção de realizar qualquer atividade de la...,Obrigações de Fazer (com custo),Reflorestamento ou recomposição vegetal,"Multa de R$ 10.000,00 para cada inobservância,...",2369738039,http://jud-anexos.digesto.com.br/09384bc8a7a2b...
